In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
import keras_tuner as kt
from keras_tuner import RandomSearch
# from GEH_LOSS import GEH_loss
import geh as ga

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [4]:
import relative_accuracy as ra

In [5]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5_multi.npz")['x'] 
Output_train_5 = np.load("Deep_train_5_multi.npz")['y']

Deep_test_5   = np.load("Deep_test_5_multi.npz")['x'] 
Output_test_5 = np.load("Deep_test_5_multi.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15_multi.npz")['x'] 
Output_train_15 = np.load("Deep_train_15_multi.npz")['y']

Deep_test_15   = np.load("Deep_test_15_multi.npz")['x'] 
Output_test_15 = np.load("Deep_test_15_multi.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30_multi.npz")['x'] 
Output_train_30 = np.load("Deep_train_30_multi.npz")['y']

Deep_test_30   = np.load("Deep_test_30_multi.npz")['x'] 
Output_test_30 = np.load("Deep_test_30_multi.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60_multi.npz")['x'] 
Output_train_60 = np.load("Deep_train_60_multi.npz")['y']

Deep_test_60   = np.load("Deep_test_60_multi.npz")['x'] 
Output_test_60 = np.load("Deep_test_60_multi.npz")['y']

In [6]:
# Wide Data
Xtrain = joblib.load("002weeks_train_Multi.save") 
Xtest = joblib.load("002weeks_test_Multi.save") 

In [7]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

In [8]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

In [9]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

In [10]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

In [11]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

C:\Users\mesug\anaconda3\envs\my_env\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

In [13]:
from pyts.image import GramianAngularField

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [14]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 7                # No of loop detectors
n_seq = 3                     # Subsequences
n_steps = 5                   # time-step per subsequence
# val_percent = 0.07567       # 2 weeks
val_percent = 0.2        
batch_size = 32
n_GAF = 15

n_seq_ = 3                     # Subsequences
n_steps_ = 3                   # time-step per subsequence

In [15]:
# Summation
gasf = GramianAngularField(image_size=1., method='summation', sample_range=None)
# Difference
gadf = GramianAngularField(image_size=1., method='difference', sample_range=None)

#### 5 mins

In [16]:
# Train
Xt5 = np.empty((len(Deep_train_5), n_features, n_GAF, n_GAF))
yt5 = scaler.inverse_transform(Output_train_5)

In [17]:
# Test
Xv5 = np.empty((len(Deep_test_5), n_features, n_GAF, n_GAF))
yv5 = scaler.inverse_transform(Output_test_5)

#### 15 mins

In [18]:
# Train
Xt15 = np.empty((len(Deep_train_15), n_features, n_GAF, n_GAF))
yt15 = scaler.inverse_transform(Output_train_15)

In [19]:
# Test
Xv15 = np.empty((len(Deep_test_15), n_features, n_GAF, n_GAF))
yv15 = scaler.inverse_transform(Output_test_15)

#### 30 mins

In [20]:
# Train
Xt30 = np.empty((len(Deep_train_30), n_features, n_GAF, n_GAF))
yt30 = scaler.inverse_transform(Output_train_30)

In [21]:
# Test
Xv30 = np.empty((len(Deep_test_30), n_features, n_GAF, n_GAF))
yv30 = scaler.inverse_transform(Output_test_30)

#### 60 mins

In [22]:
# Train
Xt60 = np.empty((len(Deep_train_60), n_features, n_GAF, n_GAF))
yt60 = scaler.inverse_transform(Output_train_60)

In [23]:
# Test
Xv60 = np.empty((len(Deep_test_60), n_features, n_GAF, n_GAF))
yv60 = scaler.inverse_transform(Output_test_60)

In [24]:
## Check Size

In [25]:
Xt5.shape

(31089, 7, 15, 15)

## Tranformations

### 5 mins

In [26]:
for i in range(len(Xt5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_5[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt5[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [27]:
for i in range(len(Xv5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_5[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv5[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [28]:
Xt5.shape

(31089, 7, 15, 15)

In [29]:
Xv5.shape

(4017, 7, 15, 15)

### 15 mins

In [30]:
for i in range(len(Xt15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_15[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt15[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [31]:
for i in range(len(Xv15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_15[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv15[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [32]:
Xt15.shape

(31087, 7, 15, 15)

In [33]:
Xv15.shape

(4015, 7, 15, 15)

### 30 mins

In [34]:
for i in range(len(Xt30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_30[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt30[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [35]:
for i in range(len(Xv30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_30[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv30[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [36]:
Xt30.shape

(31084, 7, 15, 15)

In [37]:
Xv30.shape

(4012, 7, 15, 15)

### 60 mins

In [38]:
for i in range(len(Xt60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_60[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xt30[i,:,:,:] = X_gasf
    # Xt5[i,:,:,:] = X_gadf

In [39]:
for i in range(len(Xv60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_60[i])
    
    # Perform the GAF transformations
    X_gasf = gasf.transform(X)
    # X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    Xv60[i,:,:,:] = X_gasf
    # Xv5[i,:,:,:] = X_gadf

In [40]:
Xt60.shape

(31078, 7, 15, 15)

In [41]:
Xv60.shape

(4006, 7, 15, 15)

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [42]:
# Train data
Xt5  =  np.transpose(Xt5, (0, 2, 3, 1))
Xt15 =  np.transpose(Xt15, (0, 2, 3, 1))
Xt30 =  np.transpose(Xt30, (0, 2, 3, 1))
Xt60 =  np.transpose(Xt60, (0, 2, 3, 1))

# Validation data
Xv5  =  np.transpose(Xv5, (0, 2, 3, 1))
Xv15 =  np.transpose(Xv15, (0, 2, 3, 1))
Xv30 =  np.transpose(Xv30, (0, 2, 3, 1))
Xv60 =  np.transpose(Xv60, (0, 2, 3, 1))

In [43]:
Xt5.shape

(31089, 15, 15, 7)

In [44]:
Xv5.shape

(4017, 15, 15, 7)

In [45]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_GAF, n_features)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_GAF, n_features)

In [46]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

In [47]:
# Train data
Wt5  =  Wt5.reshape(Wt5.shape[0], n_seq_, n_steps_, n_features)
Wt15 =  Wt15.reshape(Wt15.shape[0], n_seq_, n_steps_, n_features)
Wt30 =  Wt30.reshape(Wt30.shape[0], n_seq_, n_steps_, n_features)
Wt60 =  Wt60.reshape(Wt60.shape[0], n_seq_, n_steps_, n_features)

# Validation data
Wv5  =  Wv5.reshape(Wv5.shape[0], n_seq_, n_steps_, n_features)
Wv15 =  Wv15.reshape(Wv15.shape[0], n_seq_, n_steps_, n_features)
Wv30 =  Wv30.reshape(Wv30.shape[0], n_seq_, n_steps_, n_features)
Wv60 =  Wv60.reshape(Wv60.shape[0], n_seq_, n_steps_, n_features)

In [48]:
Xt5.shape

(31089, 3, 5, 15, 7)

In [49]:
Wt5.shape

(31089, 3, 3, 7)

# CNN-LSTM model

In [50]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights = True)

In [51]:
Wide_train_5.shape

(31089, 9, 7)

# 5 mins ahead

In [52]:
def build_model_5(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt5.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt5.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_5w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_5 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_5w)
    
    
    # Deep Model
    # CNN 
    CNN_5 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_5 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_5)
    # LSTM          
    LSTM_5 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_5)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_5, LSTM_5], axis=1)

    # Attention
    Att_5 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)
    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    #print(output.shape)
    
    # Model
    model_5 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_5.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                                   
    return model_5

In [53]:
tuner5 = RandomSearch(
    build_model_5,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10a-5",
)

INFO:tensorflow:Reloading Tuner from C:\Runs\10a-5\tuner0.json


In [54]:
tuner5.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [55]:
tuner5.search((Wt5,Xt5), yt5, epochs=200,
           validation_split = val_percent,
           callbacks = [early_stop],
           verbose=2)

Trial 10 Complete [00h 20m 32s]
val_loss: 619.9215698242188

Best val_loss So Far: 600.4235534667969
Total elapsed time: 12h 50m 20s
INFO:tensorflow:Oracle triggered exit


In [56]:
tuner5.results_summary(1)

Results summary
Results in C:\Runs\10a-5
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 00 summary
Hyperparameters:
units: 72
filters: 96
kernel_size: 3
learning_rate: 0.001
Score: 600.4235534667969


### Best Model

In [58]:
best_hp5 = tuner5.get_best_hyperparameters()[0]
model5 = tuner5.hypermodel.build(best_hp5)

In [59]:
scaler_filename = "Best_HP/10a-5"
joblib.dump(best_hp5, scaler_filename) 

['Best_HP/10a-5']

In [60]:
scaler_filename = "Best_HP/10a-5"
best_hp5 = joblib.load(scaler_filename) 

model5 = tuner5.hypermodel.build(best_hp5)

In [61]:
model5.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 96) 6144        DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [62]:
model5.fit((Wt5,Xt5), yt5, validation_split = val_percent,
           epochs = 100, 
           verbose = 1)

Epoch 1/100
778/778 [==============================] - 13s 11ms/step - loss: 22486.2793 - mean_absolute_error: 113.6391 - root_mean_squared_error: 149.9543 - mean_absolute_percentage_error: 56427040.0000 - val_loss: 5655.7632 - val_mean_absolute_error: 53.7114 - val_root_mean_squared_error: 75.2048 - val_mean_absolute_percentage_error: 4655168.0000
Epoch 2/100
778/778 [==============================] - 8s 11ms/step - loss: 2928.7363 - mean_absolute_error: 36.9108 - root_mean_squared_error: 54.1178 - mean_absolute_percentage_error: 60046716.0000 - val_loss: 1812.6003 - val_mean_absolute_error: 29.6229 - val_root_mean_squared_error: 42.5746 - val_mean_absolute_percentage_error: 4307955.0000
Epoch 3/100
778/778 [==============================] - 9s 11ms/step - loss: 1265.7067 - mean_absolute_error: 25.3516 - root_mean_squared_error: 35.5768 - mean_absolute_percentage_error: 59957732.0000 - val_loss: 1315.3838 - val_mean_absolute_error: 25.3872 - val_root_mean_squared_error: 36.2682 - val_

778/778 [==============================] - 40s 51ms/step - loss: 567.4253 - mean_absolute_error: 17.5836 - root_mean_squared_error: 23.8207 - mean_absolute_percentage_error: 58421144.0000 - val_loss: 804.9432 - val_mean_absolute_error: 20.0962 - val_root_mean_squared_error: 28.3715 - val_mean_absolute_percentage_error: 3733731.2500
Epoch 48/100
778/778 [==============================] - 42s 54ms/step - loss: 564.6152 - mean_absolute_error: 17.5462 - root_mean_squared_error: 23.7616 - mean_absolute_percentage_error: 58441848.0000 - val_loss: 796.4717 - val_mean_absolute_error: 19.9889 - val_root_mean_squared_error: 28.2218 - val_mean_absolute_percentage_error: 3812808.5000
Epoch 49/100
778/778 [==============================] - 39s 50ms/step - loss: 559.2304 - mean_absolute_error: 17.4812 - root_mean_squared_error: 23.6481 - mean_absolute_percentage_error: 58259936.0000 - val_loss: 791.4532 - val_mean_absolute_error: 19.9873 - val_root_mean_squared_error: 28.1328 - val_mean_absolute_per

778/778 [==============================] - 40s 52ms/step - loss: 399.3508 - mean_absolute_error: 14.9623 - root_mean_squared_error: 19.9838 - mean_absolute_percentage_error: 54082432.0000 - val_loss: 677.0504 - val_mean_absolute_error: 18.9842 - val_root_mean_squared_error: 26.0202 - val_mean_absolute_percentage_error: 3998672.2500ean_squared_error: 19.9669 - mean_absolute_percentage_err
Epoch 92/100
778/778 [==============================] - 39s 50ms/step - loss: 395.0486 - mean_absolute_error: 14.8856 - root_mean_squared_error: 19.8758 - mean_absolute_percentage_error: 53624188.0000 - val_loss: 683.6328 - val_mean_absolute_error: 19.0920 - val_root_mean_squared_error: 26.1464 - val_mean_absolute_percentage_error: 4136705.0000
Epoch 93/100
778/778 [==============================] - 39s 51ms/step - loss: 393.5710 - mean_absolute_error: 14.8569 - root_mean_squared_error: 19.8386 - mean_absolute_percentage_error: 53146468.0000 - val_loss: 693.3772 - val_mean_absolute_error: 19.2457 - val

In [63]:
metrics5 = pd.DataFrame(model5.history.history)

In [64]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics5[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics5[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(400,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [65]:
model5.save("my_dir/10a-5")

INFO:tensorflow:Assets written to: my_dir/10a-5\assets


INFO:tensorflow:Assets written to: my_dir/10a-5\assets


In [66]:
model5 = keras.models.load_model("my_dir/10a-5") 

In [67]:
model5.predict((Wv5,Xv5))

array([[ 59.02682 ,  59.79645 ,  65.27474 , ...,  64.279495,  40.046974,
         40.953808],
       [ 56.99516 ,  57.859547,  63.15796 , ...,  62.31836 ,  39.07563 ,
         39.77834 ],
       [ 55.477062,  56.19042 ,  61.280403, ...,  60.23426 ,  38.166008,
         39.09357 ],
       ...,
       [129.25044 , 128.06305 , 144.4904  , ..., 140.35992 ,  93.890236,
         87.092094],
       [123.62173 , 120.77094 , 138.22844 , ..., 132.51044 ,  86.26104 ,
         78.78473 ],
       [116.00847 , 114.18362 , 130.83052 , ..., 127.21562 ,  84.34516 ,
         81.040596]], dtype=float32)

# 15 mins ahead

In [70]:
def build_model_15(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt15.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt15.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_15w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_15 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_15w)
    
    
    
    # Deep Model
    # CNN 
    CNN_15 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_15 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_15)
    # LSTM          
    LSTM_15 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_15)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_15, LSTM_15], axis=1)

    # Attention
    Att_15 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_15 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_15.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_15

In [71]:
tuner15 = RandomSearch(
    build_model_15,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10a-15",
)

In [72]:
tuner15.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [73]:
tuner15.search((Wt15,Xt15), yt15, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop], 
            verbose=2)

Trial 10 Complete [00h 49m 26s]
val_loss: 855.87451171875

Best val_loss So Far: 747.3331909179688
Total elapsed time: 10h 02m 14s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [74]:
tuner15.results_summary(1)

Results summary
Results in C:\Runs\10a-15
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 06 summary
Hyperparameters:
units: 73
filters: 32
kernel_size: 3
learning_rate: 0.001
Score: 747.3331909179688


### Best model

In [75]:
best_hp15 = tuner15.get_best_hyperparameters()[0]
model15 = tuner15.hypermodel.build(best_hp15)

In [76]:
scaler_filename = "Best_HP/10a-15"
joblib.dump(best_hp15, scaler_filename) 

['Best_HP/10a-15']

In [77]:
scaler_filename = "Best_HP/10a-15"
best_hp15 = joblib.load(scaler_filename) 

model15 = tuner15.hypermodel.build(best_hp15)

In [78]:
model15.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 32) 2048        DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [79]:
model15.fit((Wt15,Xt15), yt15, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
778/778 [==============================] - 37s 36ms/step - loss: 22638.8047 - mean_absolute_error: 113.5351 - root_mean_squared_error: 150.4619 - mean_absolute_percentage_error: 57702084.0000 - val_loss: 5595.3398 - val_mean_absolute_error: 53.3831 - val_root_mean_squared_error: 74.8020 - val_mean_absolute_percentage_error: 4627006.0000n_absolute_error: 147.5866 - root_mean_squared_error: 18 - ETA: 7s - loss: 29394.0977 - mean_a
Epoch 2/100
778/778 [==============================] - 28s 36ms/step - loss: 2882.4387 - mean_absolute_error: 36.6689 - root_mean_squared_error: 53.6883 - mean_absolute_percentage_error: 59866736.0000 - val_loss: 1790.3788 - val_mean_absolute_error: 29.5845 - val_root_mean_squared_error: 42.3129 - val_mean_absolute_percentage_error: 4318664.0000
Epoch 3/100
778/778 [==============================] - 25s 32ms/step - loss: 1249.5582 - mean_absolute_error: 25.2649 - root_mean_squared_error: 35.3491 - mean_absolute_percentage_error: 59796068.0000 - val_

778/778 [==============================] - 25s 32ms/step - loss: 598.6176 - mean_absolute_error: 18.0397 - root_mean_squared_error: 24.4667 - mean_absolute_percentage_error: 55751300.0000 - val_loss: 801.6315 - val_mean_absolute_error: 20.2148 - val_root_mean_squared_error: 28.3131 - val_mean_absolute_percentage_error: 3876536.2500
Epoch 43/100
778/778 [==============================] - 24s 31ms/step - loss: 595.9752 - mean_absolute_error: 17.9908 - root_mean_squared_error: 24.4126 - mean_absolute_percentage_error: 55989260.0000 - val_loss: 791.2432 - val_mean_absolute_error: 20.2332 - val_root_mean_squared_error: 28.1291 - val_mean_absolute_percentage_error: 3803970.2500
Epoch 44/100
778/778 [==============================] - 25s 32ms/step - loss: 590.7324 - mean_absolute_error: 17.9230 - root_mean_squared_error: 24.3050 - mean_absolute_percentage_error: 55109312.0000 - val_loss: 768.2452 - val_mean_absolute_error: 20.0050 - val_root_mean_squared_error: 27.7172 - val_mean_absolute_per

778/778 [==============================] - 36s 46ms/step - loss: 452.4526 - mean_absolute_error: 15.9393 - root_mean_squared_error: 21.2709 - mean_absolute_percentage_error: 32015770.0000 - val_loss: 730.0443 - val_mean_absolute_error: 19.7286 - val_root_mean_squared_error: 27.0193 - val_mean_absolute_percentage_error: 4224599.0000
Epoch 85/100
778/778 [==============================] - 35s 46ms/step - loss: 448.8951 - mean_absolute_error: 15.8893 - root_mean_squared_error: 21.1871 - mean_absolute_percentage_error: 30058482.0000 - val_loss: 725.2624 - val_mean_absolute_error: 19.7072 - val_root_mean_squared_error: 26.9307 - val_mean_absolute_percentage_error: 3783274.0000r: 15.8518 - root_mean_squared_error: 21.1176 -  - ETA: 1s - loss: 448.2180 - mean_absolute_error: 15.8782 - root_mean_squared_error: 21.1712 - mean_absolute_percentage_er
Epoch 86/100
778/778 [==============================] - 36s 46ms/step - loss: 447.5816 - mean_absolute_error: 15.8673 - root_mean_squared_error: 21.

In [80]:
metrics15 = pd.DataFrame(model15.history.history)

In [81]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics15[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics15[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [82]:
model15.save("my_dir/10a-15")

INFO:tensorflow:Assets written to: my_dir/10a-15\assets


INFO:tensorflow:Assets written to: my_dir/10a-15\assets


In [83]:
model15 = keras.models.load_model("my_dir/10a-15")

In [84]:
model15.predict((Wv15,Xv15))

array([[ 57.93532 ,  59.859737,  65.97726 , ...,  64.92633 ,  40.03468 ,
         39.719524],
       [ 57.74734 ,  59.614666,  66.033005, ...,  65.0835  ,  40.406963,
         40.270184],
       [ 57.95614 ,  59.607716,  66.17795 , ...,  65.34059 ,  40.02831 ,
         40.44447 ],
       ...,
       [133.25099 , 136.6567  , 149.34543 , ..., 145.65353 ,  97.77357 ,
         92.50432 ],
       [132.18394 , 134.61342 , 148.07689 , ..., 145.83757 ,  97.26371 ,
         92.52729 ],
       [122.100586, 123.83324 , 137.25868 , ..., 135.76941 ,  88.382866,
         83.8345  ]], dtype=float32)

# 30 mins ahead

In [87]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt30.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_30w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_30 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_30w)
    
    
    # Deep Model
    # CNN 
    CNN_30 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30)
    # LSTM          
    LSTM_30 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_30)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_30, LSTM_30], axis=1)

    # Attention
    Att_30 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_30

In [88]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10a-30",
)

In [89]:
tuner30.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [90]:
tuner30.search((Wt30,Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [00h 11m 57s]
val_loss: 1137.7197265625

Best val_loss So Far: 984.0860290527344
Total elapsed time: 10h 10m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [91]:
tuner30.results_summary(1)

Results summary
Results in C:\Runs\10a-30
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
units: 72
filters: 512
kernel_size: 3
learning_rate: 0.0001
Score: 984.0860290527344


### Best model

In [92]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [93]:
scaler_filename = "Best_HP/10a-30"
joblib.dump(best_hp30, scaler_filename) 

['Best_HP/10a-30']

In [94]:
scaler_filename = "Best_HP/10a-30"
best_hp30 = joblib.load(scaler_filename) 

model30 = tuner30.hypermodel.build(best_hp30)

In [95]:
model30.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 512 32768       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [96]:
model30.fit((Wt30,Xt30), yt30, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
778/778 [==============================] - 68s 79ms/step - loss: 53710.0820 - mean_absolute_error: 201.5426 - root_mean_squared_error: 231.7544 - mean_absolute_percentage_error: 10731697.0000 - val_loss: 47419.1758 - val_mean_absolute_error: 189.4015 - val_root_mean_squared_error: 217.7595 - val_mean_absolute_percentage_error: 1659061.5000
Epoch 2/100
778/778 [==============================] - 57s 73ms/step - loss: 41257.0820 - mean_absolute_error: 170.0495 - root_mean_squared_error: 203.1185 - mean_absolute_percentage_error: 34897200.0000 - val_loss: 36641.1016 - val_mean_absolute_error: 160.4463 - val_root_mean_squared_error: 191.4186 - val_mean_absolute_percentage_error: 3462613.2500
Epoch 3/100
778/778 [==============================] - 58s 74ms/step - loss: 32208.1504 - mean_absolute_error: 146.0748 - root_mean_squared_error: 179.4663 - mean_absolute_percentage_error: 57287872.0000 - val_loss: 28586.6719 - val_mean_absolute_error: 138.6699 - val_root_mean_squared_error

778/778 [==============================] - 57s 74ms/step - loss: 834.3495 - mean_absolute_error: 21.1160 - root_mean_squared_error: 28.8851 - mean_absolute_percentage_error: 60697564.0000 - val_loss: 1168.7664 - val_mean_absolute_error: 23.6337 - val_root_mean_squared_error: 34.1872 - val_mean_absolute_percentage_error: 4155167.2500
Epoch 35/100
778/778 [==============================] - 61s 78ms/step - loss: 831.9349 - mean_absolute_error: 21.0821 - root_mean_squared_error: 28.8433 - mean_absolute_percentage_error: 60468516.0000 - val_loss: 1167.4814 - val_mean_absolute_error: 23.6405 - val_root_mean_squared_error: 34.1684 - val_mean_absolute_percentage_error: 4163171.5000
Epoch 36/100
778/778 [==============================] - 58s 74ms/step - loss: 829.0396 - mean_absolute_error: 21.0536 - root_mean_squared_error: 28.7931 - mean_absolute_percentage_error: 60510808.0000 - val_loss: 1162.0317 - val_mean_absolute_error: 23.5593 - val_root_mean_squared_error: 34.0886 - val_mean_absolute_

778/778 [==============================] - 58s 74ms/step - loss: 720.3133 - mean_absolute_error: 19.6339 - root_mean_squared_error: 26.8386 - mean_absolute_percentage_error: 59495440.0000 - val_loss: 1081.3002 - val_mean_absolute_error: 22.8218 - val_root_mean_squared_error: 32.8831 - val_mean_absolute_percentage_error: 3894964.0000
Epoch 75/100
778/778 [==============================] - 57s 73ms/step - loss: 718.2810 - mean_absolute_error: 19.6046 - root_mean_squared_error: 26.8008 - mean_absolute_percentage_error: 59413160.0000 - val_loss: 1065.9586 - val_mean_absolute_error: 22.5656 - val_root_mean_squared_error: 32.6490 - val_mean_absolute_percentage_error: 3873676.7500
Epoch 76/100
778/778 [==============================] - 58s 75ms/step - loss: 714.8368 - mean_absolute_error: 19.5580 - root_mean_squared_error: 26.7364 - mean_absolute_percentage_error: 59393380.0000 - val_loss: 1057.5657 - val_mean_absolute_error: 22.4768 - val_root_mean_squared_error: 32.5202 - val_mean_absolute_

In [97]:
metrics30 = pd.DataFrame(model30.history.history)

In [98]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics30[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics30[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [99]:
model30.save("my_dir/10a-30")

INFO:tensorflow:Assets written to: my_dir/10a-30\assets


INFO:tensorflow:Assets written to: my_dir/10a-30\assets


In [100]:
model30 = keras.models.load_model("my_dir/10a-30")

In [101]:
model30.predict((Wv30,Xv30))

array([[ 54.21254 ,  55.082134,  60.729515, ...,  59.44064 ,  36.773853,
         37.160862],
       [ 51.966225,  52.86063 ,  58.438805, ...,  57.22533 ,  35.85191 ,
         36.331104],
       [ 53.10244 ,  54.07008 ,  59.845627, ...,  58.81686 ,  37.419804,
         37.635464],
       ...,
       [143.85712 , 147.02951 , 156.04454 , ..., 155.45277 , 100.35548 ,
         93.45484 ],
       [134.54543 , 136.9028  , 146.34138 , ..., 147.71202 ,  93.69557 ,
         87.114296],
       [124.06637 , 126.31295 , 135.74872 , ..., 136.3805  ,  88.44228 ,
         82.40748 ]], dtype=float32)

# 60 mins ahead

In [104]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt60.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_60w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_60 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_60w)
    
    
    # Deep Model
    # CNN 
    CNN_60 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60)
    # LSTM          
    LSTM_60 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_60)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_60, LSTM_60], axis=1)

    # Attention
    Att_60 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [105]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10a-60",
)

In [106]:
tuner60.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [107]:
tuner60.search((Wt60,Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [00h 31m 24s]
val_loss: 1137.6492919921875

Best val_loss So Far: 1137.6492919921875
Total elapsed time: 12h 19m 25s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [108]:
tuner60.results_summary(1)

Results summary
Results in C:\Runs\10a-60
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 09 summary
Hyperparameters:
units: 27
filters: 288
kernel_size: 3
learning_rate: 0.01
Score: 1137.6492919921875


### Best model

In [109]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [110]:
scaler_filename = "Best_HP/10a-60"
joblib.dump(best_hp60, scaler_filename) 

['Best_HP/10a-60']

In [111]:
scaler_filename = "Best_HP/10a-60"
best_hp60 = joblib.load(scaler_filename) 

model60 = tuner60.hypermodel.build(best_hp60)

In [112]:
model60.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 288 18432       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [113]:
model60.fit((Wt60,Xt60), yt60, validation_split = val_percent,
            epochs = 100, 
            verbose = 1
            )

Epoch 1/100
777/777 [==============================] - 65s 74ms/step - loss: 7697.1631 - mean_absolute_error: 52.5665 - root_mean_squared_error: 87.7335 - mean_absolute_percentage_error: 62097076.0000 - val_loss: 1337.0981 - val_mean_absolute_error: 25.6524 - val_root_mean_squared_error: 36.5664 - val_mean_absolute_percentage_error: 4878527.0000
Epoch 2/100
777/777 [==============================] - 54s 69ms/step - loss: 927.3624 - mean_absolute_error: 22.3286 - root_mean_squared_error: 30.4526 - mean_absolute_percentage_error: 62813252.0000 - val_loss: 1257.4464 - val_mean_absolute_error: 24.5371 - val_root_mean_squared_error: 35.4605 - val_mean_absolute_percentage_error: 4588735.0000
Epoch 3/100
777/777 [==============================] - 55s 71ms/step - loss: 877.1278 - mean_absolute_error: 21.6953 - root_mean_squared_error: 29.6163 - mean_absolute_percentage_error: 61273608.0000 - val_loss: 1224.7526 - val_mean_absolute_error: 24.2429 - val_root_mean_squared_error: 34.9965 - val_mea

777/777 [==============================] - 55s 71ms/step - loss: 748.0656 - mean_absolute_error: 19.9358 - root_mean_squared_error: 27.3508 - mean_absolute_percentage_error: 57850196.0000 - val_loss: 1161.2623 - val_mean_absolute_error: 23.2154 - val_root_mean_squared_error: 34.0773 - val_mean_absolute_percentage_error: 4249039.0000
Epoch 46/100
777/777 [==============================] - 57s 74ms/step - loss: 749.7981 - mean_absolute_error: 19.9761 - root_mean_squared_error: 27.3824 - mean_absolute_percentage_error: 57444532.0000 - val_loss: 1141.4374 - val_mean_absolute_error: 22.9332 - val_root_mean_squared_error: 33.7852 - val_mean_absolute_percentage_error: 4342391.5000
Epoch 47/100
777/777 [==============================] - 58s 75ms/step - loss: 748.5167 - mean_absolute_error: 19.9485 - root_mean_squared_error: 27.3590 - mean_absolute_percentage_error: 57989308.0000 - val_loss: 1152.7325 - val_mean_absolute_error: 23.1454 - val_root_mean_squared_error: 33.9519 - val_mean_absolute_

777/777 [==============================] - 51s 66ms/step - loss: 733.4724 - mean_absolute_error: 19.7265 - root_mean_squared_error: 27.0827 - mean_absolute_percentage_error: 57513380.0000 - val_loss: 1158.0088 - val_mean_absolute_error: 23.1232 - val_root_mean_squared_error: 34.0295 - val_mean_absolute_percentage_error: 4447732.0000
Epoch 92/100
777/777 [==============================] - 52s 66ms/step - loss: 732.2618 - mean_absolute_error: 19.7102 - root_mean_squared_error: 27.0603 - mean_absolute_percentage_error: 57397796.0000 - val_loss: 1146.8992 - val_mean_absolute_error: 23.0014 - val_root_mean_squared_error: 33.8659 - val_mean_absolute_percentage_error: 4277010.5000
Epoch 93/100
777/777 [==============================] - 52s 67ms/step - loss: 733.6927 - mean_absolute_error: 19.7224 - root_mean_squared_error: 27.0868 - mean_absolute_percentage_error: 57408008.0000 - val_loss: 1126.4890 - val_mean_absolute_error: 22.8707 - val_root_mean_squared_error: 33.5632 - val_mean_absolute_

In [114]:
metrics60 = pd.DataFrame(model60.history.history)

In [115]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics60[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics60[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [116]:
model60.save("my_dir/10a-60")

INFO:tensorflow:Assets written to: my_dir/10a-60\assets


INFO:tensorflow:Assets written to: my_dir/10a-60\assets


In [117]:
model60 = keras.models.load_model("my_dir/10a-60")

In [118]:
model60.predict((Wv60,Xv60))

array([[ 50.806084,  52.57669 ,  58.162888, ...,  55.658173,  34.18853 ,
         32.66595 ],
       [ 51.767876,  53.36006 ,  59.14941 , ...,  56.383972,  35.09823 ,
         33.7865  ],
       [ 49.692398,  51.361984,  56.775497, ...,  55.296703,  34.078228,
         32.786762],
       ...,
       [137.15553 , 142.36897 , 152.10564 , ..., 148.57814 ,  96.28397 ,
         88.14984 ],
       [128.92668 , 134.31769 , 142.95769 , ..., 140.7562  ,  92.04637 ,
         84.84946 ],
       [122.21385 , 125.68692 , 133.41643 , ..., 128.0332  ,  84.735916,
         78.06212 ]], dtype=float32)

## Save Loss

In [ ]:
# metrics5.to_csv('Loss/8a-5.csv') 
# metrics15.to_csv('Loss/8a-15.csv') 
# metrics30.to_csv('Loss/8a-30.csv') 
# metrics60.to_csv('Loss/8a-60.csv') 